In [7]:
import pandas as pd

import gc
####

import seaborn as sns
import plotly.express as px

import numpy as np

import sqlite3
from sqlite3 import Error
import os
import re, string
from nltk.corpus import stopwords
import nltk
import matplotlib.pyplot as plt

import torch
#from transformers import AutoTokenizer, BertForSequenceClassification
## felipe


from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.decomposition import PCA

nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')


import pandas as pd
import re, string


pd.options.display.max_columns = None

#from sentence_transformers import SentenceTransformer, util
#from transformers import *

import spacy

nlp = spacy.load('es_core_news_sm', disable=['tagger', 'parser', 'ner'])



from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import preprocessing

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


# etl

In [6]:
import pandas as pd
import sqlite3

In [8]:


query1 = "SELECT NUMERO_PQR,DESCRIPCION_PQR FROM pqr_priorizar where MEDIO_RECEPCION !=  'VIRTUAL' and unico=1 and SINTOMA_PQR ='CONCEPTOS FACTURADOS' and FECHA_CREACION > '2023-06-14' "

query2 = "(SELECT * FROM altas where ST_TIPO_SERVICIO='FTTH' AND FASE='ORDERF' AND (tipoTramite= 'VTA' OR tipoTramite=  'Sales')) AS myquery"

In [9]:

# Connect to the SQLite database file
conn = sqlite3.connect('/content/on2/pqr_priorizar.db')

# Read the SQL query into a pandas DataFrame

pqrs = pd.read_sql_query(query1, conn)

# Close the database connection
conn.close()

# Print the DataFrame
pqrs

#MDM-PQR-39191683

,NUMERO_PQR,DESCRIPCION_PQR
0,MDM-PQR-39379443,SE COMUNICA POR CONCEPTOS FACTURABLES Y VALOR ...
1,MDM-PQR-39139297,BLANCA LILIA PAMPLONA PULIDO 52497227
2,MDM-PQR-39390455,error
3,MDM-PQR-39393118,error
4,MDM-PQR-39394683,cliente se comunica ya que desea confirmar por...
...,...,...
2263,MDM-PQR-39485432,Linea: 6017629943 Nombre de quien se comunic...
2264,MDM-PQR-39485464,CLIENTE INDICA QUE LE OFRECIERON UN DESCUENTO ...
2265,MDM-PQR-39485583,LTE// CLIENTE SE COMUNICA PARA VALIDAR PAGO
2266,MDM-PQR-39485702,LTE// El cliente se comunica que no le func...


# TRASNFORMAR LAS DATOS DE TEXTO

In [10]:
sys.path.append('../librerias')

In [11]:
from clean_text import clean_text

In [12]:
pqrs = pqrs.fillna("vacion")



In [13]:
pqrs.isnull().sum()

NUMERO_PQR         0
DESCRIPCION_PQR    0
dtype: int64

In [14]:
#declare answers and questions
descripcion=pqrs.DESCRIPCION_PQR





#clean_descripcion=[k for k in clean_descripcion if 'soporte'  in k]
clean_descripcion = [clean_text(des) for des in descripcion]


len(clean_descripcion)

2268

In [17]:
clean_descripcion[190]

'nonbre   blanca yaneth torres rubio   cedula correo    fijo cel      dire ion  cl c  a  in  ap  cliente se ccomunica por que quiere saber que fue lo que paso con lña factura'

spelling

In [19]:
from spelling import correct_sentence
from tqdm import tqdm

In [20]:
corrected_sentences = []
for text in tqdm(clean_descripcion, desc="Processing"):
    corrected_sentence = correct_sentence(text)
    corrected_sentences.append(corrected_sentence)

clean_descripcion=corrected_sentences

Processing:   3%|▎         | 71/2268 [00:43<30:08,  1.21it/s]

lemma token stemm

In [75]:
from lemastem import data_preprocessing

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [76]:

clean_descripcion=data_preprocessing(clean_descripcion)
clean_descripcion[3]

['pago', 'vez']

In [84]:
clean_descripcion[0]

['ganarar',
 'arena',
 'contacto',
 'dirar',
 'tv',
 'g',
 'concepto',
 'fracturado',
 'querer',
 'saber',
 'porq',
 'estar',
 'cobrar',
 'factura',
 'mil',
 'peso',
 'si',
 'pago',
 'mil']

In [91]:
merged_data=[]
for i in range(len(clean_descripcion)):
    merged_data.append(" ".join(clean_descripcion[i]))

merged_df=pd.DataFrame(merged_data,columns =['descripcion'])


In [93]:
from unidecode import unidecode

def remove_accents(text):
    return unidecode(text)

# Aplicar la función a la columna 'Texto'
merged_df['descripcion'] = merged_df['descripcion'].apply(remove_accents)


clean_descripcion = [text for text in clean_descripcion if "svas" not in text and "directv" not in text  and "winsport" not in text and "hbo" not in text  ]

len(clean_descripcion)

In [94]:
from tfidf import TFIDF

In [98]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split



X = merged_df['descripcion']
X_test=merged_df['descripcion']


X_train,_ = TFIDF(X,X_test)

# Create a new instance of PCA with the optimal number of components

pca = joblib.load('/content/PRIORIZAR/modelo/pca_binario.joblib')

#pca = PCA(n_components=585)


X_train_new = pca.fit_transform(X_train)


print("train with old features: ",np.array(X).shape)
print("train with new features:" ,np.array(X_train_new).shape)



tf-idf with 2371 features
train with old features:  (4372,)
train with new features: (4372, 585)


# cargar modelo

In [100]:
from joblib import dump

import joblib


model = joblib.load('/content/modelo_binario.joblib')

print(model)

LGBMClassifier(bagging_fraction=0.9047893258941638, bagging_freq=3,
               feature_fraction=0.8511162084506619,
               lambda_l1=0.4083871620051236, lambda_l2=0.7126556661611142,
               learning_rate=0.07376185430544126, min_child_samples=97,
               n_estimators=434, num_leaves=23)


In [111]:
y_pred = model.predict(X_train_new)

pqrs['prediccion_algoritmo'] = y_pred

pqrs['descripcion_clean']=merged_df['descripcion']



# Lista de palabras a filtrar
words_to_remove = ['svas', 'directv', 'paramont', 'hbo']

# Filtrar las filas del DataFrame
c_facturados_correcion = pqrs[~pqrs['descripcion_clean'].str.contains('|'.join(words_to_remove))]

c_facturados_SVAS = pqrs[pqrs['descripcion_clean'].str.contains('|'.join(words_to_remove))]



c_facturados_correcion


,NUMERO_PQR,DESCRIPCION_PQR,prediccion_algoritmo,descripcion_clean
0,MDM-PQR-39219396,REDES SOCIALES // Yanara Xareni // Facebook ...,1,ganarar arena contacto dirar tv g concepto fra...
1,MDM-PQR-39219473,SE COMUNICA POR CONCEPTOS FACTURABLES Y VALOR ...,1,concepto facturabl valor factura
2,MDM-PQR-39219514,Línea: 6016269266 Nombre de quien se comunica:...,1,desear ajuste factura fallo
3,MDM-PQR-39219572,cliente pago dos veces 2023-06-03 $ -86.890 2...,1,pago vez
4,MDM-PQR-39219625,cliente indica que le llego la factura le lleg...,1,llego factura llego mas cara
...,...,...,...,...
4367,MDM-PQR-39378972,LTE// El cliente informa que quiere validar ...,1,informar querer validar factura promocion plan
4368,MDM-PQR-39379057,LUISA FERNANDA BERMUDEZ GARAVITO// Solicita in...,0,bermuda granito solicitar informacion factura ...
4369,MDM-PQR-39379095,cliente indica que cambio de plan y le llego m...,1,cambio plan llego mas cara factura
4370,MDM-PQR-39379346,LADY MARLENE GALLEGO MONTOYA solicita val...,0,montan solicitar validar valor factura valido ...


In [113]:
import pandas as pd
from datetime import date


from datetime import datetime

# Obtener la fecha y hora actual
current_date = datetime.now()
# Get the current date
# Specify the root directory path
root_path = 'content/PRIORIZAR/SALIDAS'

# Export the DataFrame to a CSV file with the root path and current date in the filename
filename1 = f"{root_path}c_facturados_correcion_{current_date}.csv"
filename2 = f"{root_path}c_facturados_SVAS_{current_date}.csv"


c_facturados_correcion.to_csv(filename1, index=False)

c_facturados_SVAS.to_csv(filename2, index=False)




#pandas_c_F.to_csv('c_facturados.csv', index=False)

print("DONE")

DONE


# modelo retipificacion

filter data with 0 label

In [115]:
retipificacion = c_facturados_correcion[c_facturados_correcion['prediccion_algoritmo'] == "0"]
retipificacion

,NUMERO_PQR,DESCRIPCION_PQR,prediccion_algoritmo,descripcion_clean
6,MDM-PQR-39219669,SE COMUNICA QUERIENDO VALIDAR LA FACTURA POR A...,0,querer validar factura aumento valor
13,MDM-PQR-39220021,validacion de factura,0,validacion factura
14,MDM-PQR-39220147,NOMBRE: LEONCIO RODRIGUEZ VELASQUEZ CEDULA: ...,0,informacion facturacion
15,MDM-PQR-39220283,CC. 79765826 NOMBRE: EDELBERTO HERREÑO JEREZ ...,0,ver factura reconexion servicio
19,MDM-PQR-39220333,Nombre;CLAUDIA MARCELA OSORIO CHAVARRO Cedula...,0,diar quedar servicio debido fallo mas ver comp...
...,...,...,...,...
4335,MDM-PQR-39377410,Línea: SR-38547717 Nombre de quien se comunica...,0,dar descuento factura ver reflejado
4358,MDM-PQR-39378345,LTE// El cliente informa que quiere valida...,0,informar querer validar factura
4364,MDM-PQR-39378581,solicita información de factura se le explica ...,0,solicitar informacion factura explicar cobro
4368,MDM-PQR-39379057,LUISA FERNANDA BERMUDEZ GARAVITO// Solicita in...,0,bermuda granito solicitar informacion factura ...


In [116]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split



X = retipificacion['descripcion_clean']
X_test=retipificacion['descripcion_clean']


X_train,_ = TFIDF(X,X_test)

# Create a new instance of PCA with the optimal number of components

pca = joblib.load('/content/PRIORIZAR/modelo/pca_retipificacion.joblib')

#pca = PCA(n_components=361)


X_train_new = pca.fit_transform(X_train)


print("train with old features: ",np.array(X).shape)
print("train with new features:" ,np.array(X_train_new).shape)



tf-idf with 566 features
train with old features:  (725,)
train with new features: (725, 361)


In [117]:
model = joblib.load('/content/retipificacion.joblib')

print(model)

LGBMClassifier(bagging_fraction=0.6037067082231619, bagging_freq=7,
               feature_fraction=0.8675038227110754,
               lambda_l1=0.39291873964268126, lambda_l2=0.7561143520461611,
               learning_rate=0.07529945129406222, min_child_samples=61,
               n_estimators=361, num_leaves=147)


In [126]:
y_pred = model.predict(X_train_new)

retipificacion["retipificacion_algoritmo"]=y_pred

mapping = {'otros': 0, 'INCREMENTO DE TARIFA': 1, 'ASEGURAMIENTO': 2,  'BENEFICIO DE RETENCION': 3}


In [129]:
retipificacion

,NUMERO_PQR,DESCRIPCION_PQR,prediccion_algoritmo,descripcion_clean,retipificacion_algoritmo
6,MDM-PQR-39219669,SE COMUNICA QUERIENDO VALIDAR LA FACTURA POR A...,0,querer validar factura aumento valor,1
13,MDM-PQR-39220021,validacion de factura,0,validacion factura,0
14,MDM-PQR-39220147,NOMBRE: LEONCIO RODRIGUEZ VELASQUEZ CEDULA: ...,0,informacion facturacion,2
15,MDM-PQR-39220283,CC. 79765826 NOMBRE: EDELBERTO HERREÑO JEREZ ...,0,ver factura reconexion servicio,0
19,MDM-PQR-39220333,Nombre;CLAUDIA MARCELA OSORIO CHAVARRO Cedula...,0,diar quedar servicio debido fallo mas ver comp...,1
...,...,...,...,...,...
4335,MDM-PQR-39377410,Línea: SR-38547717 Nombre de quien se comunica...,0,dar descuento factura ver reflejado,1
4358,MDM-PQR-39378345,LTE// El cliente informa que quiere valida...,0,informar querer validar factura,1
4364,MDM-PQR-39378581,solicita información de factura se le explica ...,0,solicitar informacion factura explicar cobro,3
4368,MDM-PQR-39379057,LUISA FERNANDA BERMUDEZ GARAVITO// Solicita in...,0,bermuda granito solicitar informacion factura ...,3


In [130]:
import pandas as pd
from datetime import date


from datetime import datetime

# Obtener la fecha y hora actual
current_date = datetime.now()
# Get the current date
# Specify the root directory path
root_path = 'content/'

# Export the DataFrame to a CSV file with the root path and current date in the filename
filename3 = f"{root_path}c_facturados_retipificacion_{current_date}.csv"



retipificacion.to_csv(filename3, index=False)





#pandas_c_F.to_csv('c_facturados.csv', index=False)

print("DONE")

DONE
